In [193]:
import sqlite3
import pandas as pd
import numpy as np
import re

In [194]:
conn=sqlite3.connect('posts.db')

In [225]:
df=pd.read_sql_query('select * from info',con=conn)
df['option']=df['option'].apply(eval)
df=pd.concat([df.drop(['option'],axis=1),pd.io.json.json_normalize(df['option'])],axis=1)
df.columns=['id','post_id','tite','description','price','create_time','location','elevator','storage','rooms','age','ppm','meter','type','parking']
df[['state','county','region']]= df['location'].str.split('،',n=2,expand=True)
df=df.drop('location',axis=1)

In [226]:
df=df.dropna(subset=['ppm'])

In [227]:
df['price']=df['price'].str.replace("تومان","")
df['price']=df['price'].str.replace(",","")


In [228]:
df['meter']=df['meter'].str.replace(',','')
df['meter']=df['meter'].astype('int64')

In [229]:
df['ppm']=df['ppm'].str.replace(',','')
df['ppm']=df['ppm'].astype('int64')

In [230]:
df['price']=np.where(df['price']=='قیمت توافقی',df['meter']*df['ppm'],df['price'])
df['price']=df['price'].astype('float')

In [231]:
# code ha ta inja baraye soton price bood

In [232]:
df['parking']=np.where(df['parking']==' دارد ',1,0)

In [233]:
df['elevator']=np.where(df['elevator']==' دارد ',1,0)

In [234]:
df['storage']=np.where(df['storage']==' دارد ',1,0)

In [235]:
df['age']=df['age'].apply(lambda x: re.sub(r'\D','',x))
df['age'][df['age']=='']=1
df['age']=df['age'].astype('int64')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [236]:
df=df.drop('state',axis=1)

In [237]:
mapping_dict = {
    ' 1 ': 1,
    ' 2 ': 2,
    ' ۲ ': 2,
    ' 3 ': 3,
    ' ۳ ': 3,
    ' 5 به بالا ': 5,
    ' ۵ به بالا ': 5,
    ' 4 ': 4,
    ' ۴ ': 4,
    ' ۱ ': 1,
    ' بدون اتاق ': 0,
}
df['rooms']=df['rooms'].map(mapping_dict)

In [238]:
df.to_excel('house.xlsx')